In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass


In [ ]:
@dataclass
class modelconf:
  n_layer: int = 12
  n_head: int = 12
  n_embd: int = 768
  vocab_size: int = 50257
  block_size: int = 1024

class Attention(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.n_head = conf.n_head
        self.n_embd = conf.n_embd
        self.scale = (conf.n_embd // conf.n_head) ** -0.5 # scale factor
        #projection for output
        self.c_proj=nn.Linear(conf.n_embd, conf.n_embd)
        self.c_attn = nn.Linear(conf.n_embd, 3 * conf.n_embd)
        self.register_buffer('bias', torch.tril(torch.ones(conf.block_size, conf.block_size)))
    def forward(self, x):
        B, T, C = x.shape
        qkv=self.c_attn(x)
        q, k, v = qkv.split(self.n_embd, dim=2)
        k=k.view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        q=q.view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        v=v.view(B,T,self.n_head,C//self.n_head).transpose(1,2)
        att=(q @ k.transpose(-2,-1)) * self.scale
        att=att.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        att=F.softmax(att, dim=-1)
        y=att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.c_proj(out)
        return y

class FFN(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.c_fc = nn.Linear(conf.n_embd, 4 * conf.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.c_proj = nn.Linear(4 * conf.n_embd, conf.n_embd)

    def forward(self, x):
        x = self.c_fc(x)
        x = self.gelu(x)
        x = self.c_proj(x)
        return x

class Layer(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.ln_1 = nn.LayerNorm(conf.n_embd)
        self.attn = Attention(conf)
        self.ln_2 = nn.LayerNorm(conf.n_embd)
        self.mlp = FFN(conf)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x



In [ ]:
class GPT2(nn.Module):
    def __init__(self, conf):
        super().__init__()
        self.conf = conf

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(conf.vocab_size, conf.n_embd),
            wpe=nn.Embedding(conf.block_size, conf.n_embd),
            h = nn.ModuleList([Layer(conf) for _ in range(conf.n_layer)]),
            ln_f = nn.LayerNorm(conf.n_embd),
        ))
        self.lm_head = nn.Linear(conf.n_embd, conf.vocab_size, bias=False)
    @classmethod
    def from_pretrained(cls, model_type):
        """Loads pretrained GPT-2 model weights from huggingface"""
        assert model_type in {'gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl'}
        from transformers import GPT2LMHeadModel
        print("loading weights from pretrained gpt: %s" % model_type)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        # create a from-scratch initialized minGPT model
        config = modelconf(**config_args)
        model = GPT2(config)
        sd = model.state_dict()
        sd_keys = sd.keys()
        sd_keys = [k for k in sd_keys if not k.endswith('.attn.bias')] # discard this mask / buffer, not a param

        # init a huggingface/transformers model
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()

        # copy while ensuring all of the parameters are aligned and match in names and shapes
        sd_keys_hf = sd_hf.keys()
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.masked_bias')] # just a buffer
        sd_keys_hf = [k for k in sd_keys_hf if not k.endswith('.attn.bias')] # just the mask (buffer)
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        #  transpose these weights when imported
        assert len(sd_keys_hf) == len(sd_keys), f"mismatched keys: {len(sd_keys_hf)} != {len(sd_keys)}"
        for k in sd_keys_hf:
            if any(k.endswith(w) for w in transposed):
                # special treatment for the Conv1D weights we need to transpose
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                # vanilla copy over the other parameters
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])

        return model

In [ ]:
model=GPT2.from_pretrained('gpt2')
print('success')

loading weights from pretrained gpt: gpt2
success
